# GVol Historical Options Data Query

## Instructions and Description

This Python script is crafted to fetch options chain data for specific cryptocurrencies from the GVol API. Here is a detailed rundown of the code's structure and functionalities.

### 1. Importing Essential Libraries
The code begins by importing the necessary Python libraries:

- `requests`: To execute HTTP POST requests to the GVol API.
- `json`: To handle and parse JSON formatted responses from the API.
- `pandas`: For manipulating data using DataFrames, as well as for date-related calculations.

### 2. API Key and Endpoint Configuration
Two crucial variables are defined for API interaction:

- `GVOL_API_KEY`: This variable holds the key required for API authentication.
- `url`: Defines the endpoint URL for the API.

### 3. Defining the Function `fetch_options_chain`

This is the main function of the script, responsible for data retrieval, normalization, filtering based on expiration date, and optional column exclusion.

#### Parameters:

- `start_date` and `end_date`: Define the date range for which data will be fetched. Must be in 'YYYY-MM-DD' format.
- `ticker`: Stipulates the cryptocurrency of interest, such as "BTC" for Bitcoin or "ETH" for Ethereum.
- `frequency`: Sets the time interval for the data. Valid values are "1 day" or "1 hour".
- `expiration_range_in_days`: Optional. Filters out options that expire beyond this number of days from the date of query.
- `columns_to_exclude`: Optional. Provides a list of column names that will be removed from the final DataFrame.
- `exchange`: Optional. The exchange to fetch data from, defaulting to "deribit".
- `api_key`: Optional. A custom API key can be provided to override `GVOL_API_KEY`.
- `url`: Optional. A custom URL can be provided to override the default endpoint.

#### Additional Features:

- Executes API requests within a for-loop, iterating through the date range defined by `start_date` and `end_date`.
- Incorporates a filter to exclude rows with an expiration date beyond `expiration_range_in_days` from the query date.
- Aggregates the results from each API call into a single DataFrame.
- Optionally excludes columns listed in `columns_to_exclude`.

### 4. Sample Function Call and Variable Configuration

You can set up your variables like this:

```python
start_date = "2022-01-01"
end_date = "2023-01-01"
ticker = "ETH"
frequency = "1 hour"
expiration_range_in_days = 30
columns_to_exclude = ['interval','currency', 'vega', 'rho']
```

And then call the function:

```python
df = fetch_options_chain(start_date=start_date, end_date=end_date, ticker=ticker, frequency=frequency, expiration_range_in_days=expiration_range_in_days, columns_to_exclude=columns_to_exclude)
```

This call will fetch options chain data for Ethereum between January 1, 2022, and January 1, 2023, at 1-hour intervals. It will then filter out options with expiration dates beyond 3 days from the query date and exclude specified columns like 'interval', 'currency', 'vega', and 'rho'.

*Note: This code may take a while to run, as it iterates query-by-query for each day of data.*

In [1]:
# 1. Importing Essential Libraries
import requests
import json
import pandas as pd

# 2. API Key and Endpoint Configuration
GVOL_API_KEY = "YOUR_GVOL_API_KEY"
url = "https://app.pinkswantrading.com/graphql"

# 3. Defining the Function `fetch_options_chain`
def fetch_options_chain(start_date, end_date, ticker, frequency, expiration_range_in_days=None, columns_to_exclude=[], exchange="deribit", api_key=GVOL_API_KEY, url=url):
    # 3.1 Parameter Validation
    if frequency not in ["1 day", "1 hour"]:
        raise ValueError("Frequency must be either '1 day' or '1 hour'.")

    # 3.2 Initialize DataFrame and Date Range
    date_range = pd.date_range(start_date, end_date, freq='D')
    df = pd.DataFrame()

    # 3.3 Fetch Data
    for date in date_range:
        # API Query Configuration
        payload = {
            "query": "query HifiVolSurfaceStrikesGreeksHourly($symbol: BTCOrETHEnumType, $date: String, $interval:String, $exchange: ExchangeEnumType ){HifiVolSurfaceStrikesGreeksHourly(symbol: $symbol, date: $date, interval: $interval, exchange:$exchange) {date currency expiration strike instrumentName interval putCall spot underlyingPrice openInterest bidIv markIv askIv bestBidAmount bestBidPrice markPrice bestAskPrice bestAskAmount delta gamma theta vega rho } }",
            "variables": {"symbol": ticker, "date": date.strftime('%Y-%m-%d'), "interval": "1 hour", "exchange": exchange}
        }

        # API Request Headers
        headers = {
            'x-oracle': api_key,
            'Content-Type': 'application/json',
            'accept': '*/*',
            'Accept-Language': 'en-US,en;q=0.9'
        }

        # Execute API Call
        response = requests.post(url, headers=headers, json=payload)
        
        # Parse Response Data
        data = json.loads(response.text)
        chain = pd.json_normalize(data['data']['HifiVolSurfaceStrikesGreeksHourly'])
        
        # Data Transformation
        chain['date'] = pd.to_datetime(chain['date'], unit='ms')
        chain['expiration'] = pd.to_datetime(chain['expiration'], unit='ms')
        chain['interval'] = pd.to_datetime(chain['interval'], unit='ms')

        # 3.4 Filter Data Based on Expiration
        if expiration_range_in_days:
            mask = (chain['expiration'] <= (chain['date'] + pd.DateOffset(days=expiration_range_in_days)))
            chain = chain[mask]

        # 3.5 Aggregating Data
        df = pd.concat([df, chain])

    # 3.6 Column Exclusion
    df.drop(columns=columns_to_exclude, errors='ignore', inplace=True)

    return df

In [2]:
# 4. Function Call and Parameter Initialization

# 4.1 Defining Input Parameters
start_date = "2022-01-01"
end_date = "2022-01-02"
ticker = "ETH"
frequency = "1 hour"
expiration_range_in_days = 3

# 4.2 Defining Columns to Exclude
columns_to_exclude = ['interval', 'currency', 'vega', 'rho']

# 4.3 Invoking the Function
df = fetch_options_chain(start_date=start_date, end_date=end_date, ticker=ticker, frequency=frequency, expiration_range_in_days=expiration_range_in_days, columns_to_exclude=columns_to_exclude)

# 4.4 Display the DataFrame
df

,date,expiration,strike,instrumentName,putCall,spot,underlyingPrice,openInterest,bidIv,markIv,askIv,bestBidAmount,bestBidPrice,markPrice,bestAskPrice,bestAskAmount,delta,gamma,theta
0,2022-01-01 00:00:00,2022-01-01 08:00:00,4400,ETH-1JAN22-4400-C,C,3675.76,3676.98,0,0.0,100.00,275.23,0,0.0,0.000000,0.0005,382,0.00000,0.00000,0.00000
1,2022-01-01 00:00:00,2022-01-01 08:00:00,4300,ETH-1JAN22-4300-C,C,3677.42,3677.76,0,0.0,93.91,244.88,0,0.0,0.000000,0.0005,402,0.00000,0.00000,0.00000
2,2022-01-01 00:00:00,2022-01-01 08:00:00,4200,ETH-1JAN22-4200-C,C,3677.42,3677.76,0,0.0,90.16,213.48,0,0.0,0.000000,0.0005,372,0.00000,0.00000,-0.00001
3,2022-01-01 00:00:00,2022-01-01 08:00:00,4100,ETH-1JAN22-4100-C,C,3677.42,3677.76,42,0.0,84.40,180.57,0,0.0,0.000000,0.0005,362,0.00001,0.00000,-0.00022
4,2022-01-01 00:00:00,2022-01-01 08:00:00,4050,ETH-1JAN22-4050-C,C,3676.16,3676.32,171,0.0,83.33,164.00,0,0.0,0.000000,0.0005,366,0.00006,0.00000,-0.00134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11631,2022-01-02 23:00:00,2022-01-04 08:00:00,3400,ETH-4JAN22-3400-P,P,3829.04,3828.77,1,0.0,85.90,109.89,0,0.0,0.000209,0.0010,542,-0.01133,0.00015,-0.79954
11632,2022-01-02 23:00:00,2022-01-04 08:00:00,3300,ETH-4JAN22-3300-P,P,3829.04,3828.77,0,0.0,93.54,132.39,0,0.0,0.000081,0.0010,542,-0.00443,0.00006,-0.30975
11633,2022-01-02 23:00:00,2022-01-04 08:00:00,3200,ETH-4JAN22-3200-P,P,3829.04,3828.77,0,0.0,100.00,155.08,0,0.0,0.000028,0.0010,542,-0.00157,0.00002,-0.10757
11634,2022-01-02 23:00:00,2022-01-04 08:00:00,3100,ETH-4JAN22-3100-P,P,3829.04,3828.77,0,0.0,100.00,160.15,0,0.0,0.000004,0.0005,20,-0.00026,0.00000,-0.01567


### 5. Saving and Displaying the DataFrame

#### 5.1 Exporting the DataFrame to a CSV File
The code exports the DataFrame to a CSV file named `sample_btc_options_data.csv`. Update `insert_your_file_path` with your unique file path where you want to save this CSV file.


#### 5.2 Displaying the DataFrame
Finally, the DataFrame `df` containing the fetched data is displayed.



In [ ]:
# 5.1 Exporting the DataFrame to a CSV File
filename = "sample_btc_options_data"
df.to_csv(f'insert_your_file_path/{filename}.csv', index=False) # Copy and paste your unique file path

# 5.2 Display the Dataframe the DataFrame
df